## Notebook Set Up

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
print(db.list_collection_names())

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene' : 20}
results = list(establishments.find(query))
# Use count_documents to display the number of documents in the result
print(f'{establishments.count_documents(query)} establishments have a hygiene score of 20.')
# Display the first document in the results using pprint
pprint(results[0])


In [ ]:
# Convert the result to a Pandas DataFrame
import pandas as pd
hygieneDf = pd.DataFrame(results)
# Display the number of rows in the DataFrame
len(hygieneDf)
# Display the first 10 rows of the DataFrame
hygieneDf.head(10)


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName' : {'$regex' : 'London'}, {'RatingValue' : {'$gte' : 4}}}
results = list(establishments.find(query))
# Use count_documents to display the number of documents in the result
print(f'{establishments.count_documents(query)} establishments with London as the Local Authority have a rating value greater than or equal to 4')
# Display the first document in the results using pprint
pprint(results[0])


In [ ]:
# Convert the result to a Pandas DataFrame
ratingDf = pd.DataFrame(results)
# Display the number of rows in the DataFrame
len(ratingDf)
# Display the first 10 rows of the DataFrame
ratingDf.head(10)


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# to get lat and lon
establishments.find_one({'BusinessName' : 'Penang Flavours'})

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score


degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {'geocode.latitude' : {'$lt' : longitude+degree_search,'$gt' : longitude-degree_search},
        'geocode.longitude' : {'$lt' : latitude+degree_search,'$gt' : latitude-degree_search}, 'RatingValue' : 5}
sort = [('scores.Hygiene',-1)]

results = list(establishments.find(query).sort(sort).limit(5))

# Print the results
pprint(results)


In [ ]:
# Convert result to Pandas DataFrame
nearestDf = pd.DataFrame(results)
results.head()

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

query1 = {'$match' : {'scores.Hygiene' : 0}}
query2 = {'$group' : {'_id' : '$LocalAuthorityName','count' : {'$sum' : 1}}}
query3 = {'$sort' : {'count' : -1}}
pipeline = [query1, query2, query3]
results = list(establishments.aggregate(pipeline))
# Print the number of documents in the result

print(f'There are {len(results)} documents in the result')
# Print the first 10 results
pprint(results[0:10])


In [ ]:
# Convert the result to a Pandas DataFrame
inAreaDf = pd.DataFrame(results)
# Display the number of rows in the DataFrame
len(inAreaDf)
# Display the first 10 rows of the DataFrame
inAreaDf.head(10)